The `process_csv` function is responsible for processing CSV files in a given folder and creating hourly data from them. Let's go through the implementation step by step:

1. The function takes two parameters: `folder_path` (the path to the folder containing the CSV files) and `output_folder` (the path to the folder where the hourly data will be saved).

2. It uses a `for` loop to iterate over each file in the `folder_path` directory. The `os.listdir` function returns a list of all files and directories in the specified path.

3. The `if` statement checks if the file has a `.csv` extension using the `endswith` method. If it does, the file is considered a CSV file and further processing is performed.

4. The `file_path` is created by joining the `folder_path` and `file_name` using the `os.path.join` function.

5. The CSV file is read into a Pandas DataFrame using the `pd.read_csv` function, and the resulting DataFrame is stored in the `data` variable.

6. The 'datetime' column in the DataFrame is converted to datetime format using the `pd.to_datetime` function.

7. The DataFrame index is set to the '_id' column using the `data.set_index` method. This assumes that '_id' is a unique identifier for each row.

8. The data is resampled by hour using the `data.resample` method with a frequency of 'H' (hourly) and the 'datetime' column as the time reference.

9. Inside the `resample` method, a lambda function is used to calculate the weighted average price and total volume for each hour. The lambda function returns a Pandas Series with two columns: 'weighted_avg_price' and 'total_volume'. If the resampled data is empty (no data for that hour), the lambda function returns 0 for both columns.

10. The columns in the `hourly_data` DataFrame are rearranged to have 'total_volume' first using the indexing operation `hourly_data[['total_volume', 'weighted_avg_price']]`.

11. The filename without the extension is extracted using the `os.path.splitext` function.

12. An `if` statement checks if the `output_directory` (specified in the main code) exists. If it doesn't, the function creates the directory using `os.makedirs(output_directory)`.

13. The new filename is created by appending '_hourly' to the original filename, and the new file path is created by joining the `output_folder` and the new filename.

14. Another `if` statement checks if the new file path already exists. If it does, it means that hourly data for that file already exists. In this case, the existing hourly data is read from the file using `pd.read_csv` with the '_id' column set as the index.

15. The existing hourly data and the newly calculated hourly data are appended together using the `existing_hourly_data.append(hourly_data)` method.

16. The combined hourly data is then sorted by the index ('_id') using the `combined_hourly_data.sort_index` method.

17. Finally, the combined hourly data is saved to the new file path using the `combined_hourly_data.to_csv` method. If the new file path doesn't exist, indicating that hourly data for that file doesn't exist yet, the hourly data is simply saved to the new file path.


In [ ]:
import pandas as pd
import os

# Root directory containing subdirectories
root_directory = r'C:\Users\aksha\Downloads\data_eng_fin\classification'

# Function to process CSV files and create hourly data
def process_csv(folder_path, output_folder):
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.csv'):
            file_path = os.path.join(folder_path, file_name)

            # Read the CSV file into a Pandas DataFrame
            data = pd.read_csv(file_path)

            # Convert 'datetime' column to datetime format
            data['datetime'] = pd.to_datetime(data['datetime'])

            # Set '_id' as the DataFrame index (assuming it's a unique identifier)
            data.set_index('_id', inplace=True)

            # Resample the data by hour and calculate weighted average price and total volume
            hourly_data = data.resample('H', on='datetime').apply(
                lambda x: pd.Series({
                    'weighted_avg_price': (x['v'] * x['vw']).sum() / x['v'].sum() if not x.empty else 0,
                    'total_volume': x['v'].sum() if not x.empty else 0
                })
            )

            # Rearrange columns to include 'volume' first
            hourly_data = hourly_data[['total_volume', 'weighted_avg_price']]

            # Get filename without extension
            filename_without_extension, extension = os.path.splitext(file_name)
            if not os.path.exists(output_directory):
            # Create the hourly_data folder
                 os.makedirs(output_directory)
                 print(f"Created {output_directory}")

            # New filename with '_hourly' added
            new_filename = f"{filename_without_extension}_hourly{extension}"
            new_file_path = os.path.join(output_folder, new_filename)
            print(new_file_path)

            # If hourly file already exists, append the data and sort by index (_id)
            if os.path.exists(new_file_path):
                existing_hourly_data = pd.read_csv(new_file_path, index_col='_id')
                combined_hourly_data = existing_hourly_data.append(hourly_data)
                combined_hourly_data.sort_index(inplace=True)
                combined_hourly_data.to_csv(new_file_path)
            else:
                # Save the hourly data to a new CSV file
                hourly_data.to_csv(new_file_path)

# Iterate through subdirectories
historical_fx_rates_folder = os.path.join(root_directory, 'historical_fx_rates')
output_directory = os.path.join(root_directory, 'hourly_data')
print(historical_fx_rates_folder)
if os.path.exists(historical_fx_rates_folder):
    print(f"Processing {historical_fx_rates_folder}")
    process_csv(historical_fx_rates_folder, output_directory)


In [ ]:

"""
This script performs fractal dimension (FD) calculation and normalization on hourly data files.
It reads each CSV file in the input directory, calculates the FD values for each 6-hour interval,
and saves the original and normalized FD values into separate files.

Parameters:
    input_directory (str): The directory path where the input CSV files are located.
    output_directory (str): The directory path where the output files will be saved.
    new_folder_name (str): The name of the new folder to be created in the output directory.
    
Returns:
    None
"""

import os
import pandas as pd

input_directory = r'C:/Users/aksha/Downloads/data_eng_fin/classification/hourly_data'
output_directory = r'C:/Users/aksha/Downloads/data_eng_fin/classification/hourly_data/'

# Create a new folder if it doesn't exist
new_folder_name = 'fd_normalized'
new_output_directory = os.path.join(output_directory, new_folder_name)
if not os.path.exists(new_output_directory):
    os.makedirs(new_output_directory)

all_files = [os.path.join(input_directory, file) for file in os.listdir(input_directory) if file.endswith('.csv')]

for file_path in all_files:
    print(f'Operating on file: {file_path}')  # Print the operating file name
    
    df = pd.read_csv(file_path)
    df['datetime'] = pd.to_datetime(df['datetime'])
    
    file_name = os.path.splitext(os.path.basename(file_path))[0]
    
    # Initialize variables to store the mean and volatility for the first 6-hour window
    previous_mean_value = 0
    previous_volatility = 0
    
    # Lists to store FD values for normalization
    fd_values = []
    
    # Iterate through each 6-hour interval
    for idx, interval_start in enumerate(pd.date_range(df['datetime'].min(), df['datetime'].max(), freq='6H')):
        if idx == 0:
            # For the first 6-hour window, don't calculate FD
            continue
        
        interval_end = interval_start + pd.Timedelta(hours=6)
        interval_data = df[(df['datetime'] >= interval_start) & (df['datetime'] < interval_end)]
        
        # Perform calculations for the 6-hour interval using previous mean and volatility
        mean_value = previous_mean_value
        volatility = previous_volatility
        
        if len(interval_data) > 0:
            mean_value = interval_data['weighted_avg_price'].mean()
            max_price = interval_data['weighted_avg_price'].max()
            min_price = interval_data['weighted_avg_price'].min()
            
            # Avoid division by zero by checking if mean_value is not zero
            if mean_value != 0:
                volatility = (max_price - min_price) / mean_value
            else:
                volatility = 0  # Set volatility to 0 if mean_value is 0 to avoid division by zero
        
        max_min_diff = max_price - min_price if len(interval_data) > 0 else 0

        daily_crossings = 0
        for _, row in interval_data.iterrows():
            crossings = sum(((row['weighted_avg_price'] > mean_value + n * 0.025 * volatility) | (row['weighted_avg_price'] < mean_value - n * 0.025 * volatility)) for n in range(1, 101))
            daily_crossings += crossings

        fd = daily_crossings / max_min_diff if max_min_diff != 0 else 0
        fd_values.append(fd)
        
        # Update previous_mean_value and previous_volatility for the next iteration
        previous_mean_value = mean_value
        previous_volatility = volatility
    
    # Normalize FD values from 0 to 1
    min_fd = min(fd_values)
    max_fd = max(fd_values)
    normalized_fd_values = [(x - min_fd) / (max_fd - min_fd) for x in fd_values]
    
    # Create a DataFrame with original and normalized FD values for each 6-hour interval
    intervals = pd.date_range(df['datetime'].min() + pd.Timedelta(hours=6), df['datetime'].max(), freq='6H')
    fd_data = pd.DataFrame({'Interval_Start': intervals, 'Currency': [file_name] * len(intervals), 'FD_Original': fd_values, 'FD_Normalized': normalized_fd_values})
    
    # Save each currency's FD data into a separate file
    normalized_output_filename = os.path.join(new_output_directory, f'{file_name}_fractal_dimension_normalized.csv')
    fd_data.to_csv(normalized_output_filename, mode='a', header=not os.path.exists(normalized_output_filename), index=False)


In [1]:
"""
This code calculates the Fractal Dimension (FD) for each currency in a given directory of hourly data.
The FD is calculated based on the weighted average price of each currency over 6-hour intervals.
The code reads each CSV file in the input directory, performs calculations for each interval, and saves the FD data into separate files.

Parameters:
- input_directory: The directory path where the input CSV files are located.
- output_directory: The directory path where the output FD files will be saved.
- new_folder_name: The name of the new folder to be created within the output directory.
- n: The number of standard deviations used to determine the upper and lower thresholds for crossings.

Returns:
- None

"""

import os
import pandas as pd
input_directory = r'C:/Users/aksha/Downloads/data_eng_fin/classification/hourly_data'
output_directory = r'C:/Users/aksha/Downloads/data_eng_fin/classification/hourly_data/fd'

# Create a new folder if it doesn't exist
new_folder_name = 'weighted_avg_output_folder'
new_output_directory = os.path.join(output_directory, new_folder_name)
if not os.path.exists(new_output_directory):
    os.makedirs(new_output_directory)

all_files = [os.path.join(input_directory, file) for file in os.listdir(input_directory) if file.endswith('.csv')]

for file_path in all_files:
    print(f'Operating on file: {file_path}')  # Print the operating file name
    
    df = pd.read_csv(file_path)
    df['datetime'] = pd.to_datetime(df['datetime'])
    
    file_name = os.path.splitext(os.path.basename(file_path))[0]
    
    # Initialize variables to store the mean and volatility for the first 6-hour window
    previous_mean_value = 0
    previous_volatility = 0
    
    # Lists to store FD values for normalization
    fd_values = []
    min_prices = []
    max_prices = []
    volatilities = []
    vwap_values = []
    
    # Iterate through each 6-hour interval
    for idx, interval_start in enumerate(pd.date_range(df['datetime'].min(), df['datetime'].max(), freq='6H')):
        if idx == 0:
            # For the first 6-hour window, don't calculate FD
            continue
        
        interval_end = interval_start + pd.Timedelta(hours=6)
        interval_data = df[(df['datetime'] >= interval_start) & (df['datetime'] < interval_end)]
        
        # Perform calculations for the 6-hour interval using previous mean and volatility
        mean_value = previous_mean_value
        volatility = previous_volatility
        vwap = 0
        
        if len(interval_data) > 0:
            mean_value = interval_data['weighted_avg_price'].mean()
            max_price = interval_data['weighted_avg_price'].max()
            min_price = interval_data['weighted_avg_price'].min()
            
            # Calculate VWAP
            if interval_data['total_volume'].sum() != 0:  # Ensure denominator is not zero
                vwap = (interval_data['weighted_avg_price'] * interval_data['total_volume']).sum() / interval_data['total_volume'].sum()
            else:
                vwap = 0  # Set VWAP to 0 if the sum of total_volume is zero
            
            if mean_value != 0:  # Avoid division by zero for volatility calculation
                volatility = (max_price - min_price) / mean_value
            else:
                volatility = 0  # Set volatility to 0 if mean_value is 0 to avoid division by zero
        
        max_min_diff = max_price - min_price if len(interval_data) > 0 else 0

        daily_crossings = 0
        for _, row in interval_data.iterrows():
            crossings = sum(((row['weighted_avg_price'] > mean_value + n * 0.025 * volatility) | (row['weighted_avg_price'] < mean_value - n * 0.025 * volatility)) for n in range(1, 101))
            daily_crossings += crossings

        fd = daily_crossings / max_min_diff if max_min_diff != 0 else 0
        fd_values.append(fd)
        min_prices.append(min_price)
        max_prices.append(max_price)
        volatilities.append(volatility)
        vwap_values.append(vwap)
        
        # Update previous_mean_value and previous_volatility for the next iteration
        previous_mean_value = mean_value
        previous_volatility = volatility
    
    # Normalize FD values from 0 to 1
    min_fd = min(fd_values)
    max_fd = max(fd_values)
    normalized_fd_values = [(x - min_fd) / (max_fd - min_fd) for x in fd_values]
    
    # Create a DataFrame with original and normalized FD values, along with additional stats
    intervals = pd.date_range(df['datetime'].min() + pd.Timedelta(hours=6), df['datetime'].max(), freq='6H')
    fd_data = pd.DataFrame({'Interval_Start': intervals, 'FD_Original': fd_values, 'FD_Normalized': normalized_fd_values,
                            'Min_Weighted_Avg_Price': min_prices, 'Max_Weighted_Avg_Price': max_prices, 
                            'Volatility': volatilities, 'VWAP': vwap_values, 'Currency': [file_name] * len(intervals)})
    
    # Save each currency's FD data into a separate file
    normalized_output_filename = os.path.join(new_output_directory, f'{file_name}_fractal_dimension_normalized_weight.csv')
    fd_data.to_csv(normalized_output_filename, mode='a', header=not os.path.exists(normalized_output_filename), index=False)


Operating on file: C:/Users/aksha/Downloads/data_eng_fin/classification/hourly_data\AUDCAD_hourly.csv
Operating on file: C:/Users/aksha/Downloads/data_eng_fin/classification/hourly_data\AUDCHF_hourly.csv
Operating on file: C:/Users/aksha/Downloads/data_eng_fin/classification/hourly_data\AUDHKD_hourly.csv
Operating on file: C:/Users/aksha/Downloads/data_eng_fin/classification/hourly_data\AUDJPY_hourly.csv
Operating on file: C:/Users/aksha/Downloads/data_eng_fin/classification/hourly_data\AUDNZD_hourly.csv
Operating on file: C:/Users/aksha/Downloads/data_eng_fin/classification/hourly_data\AUDSGD_hourly.csv
Operating on file: C:/Users/aksha/Downloads/data_eng_fin/classification/hourly_data\CADCHF_hourly.csv
Operating on file: C:/Users/aksha/Downloads/data_eng_fin/classification/hourly_data\CADHKD_hourly.csv
Operating on file: C:/Users/aksha/Downloads/data_eng_fin/classification/hourly_data\CADJPY_hourly.csv
Operating on file: C:/Users/aksha/Downloads/data_eng_fin/classification/hourly_dat

In [2]:
from pycaret.regression import *
# Establishing path for the folder which contains all the csv files.
input_folder = "C:/Users/aksha/Downloads/data_eng_fin/classification/hourly_data/fd_normalized/weighted_avg_output_folder"

# Get a list of all csv files in the input folder
csv_files = [csv_file for csv_file in os.listdir(input_folder) if csv_file.endswith('.csv')]
df_list = []

for csv_file in csv_files:
    # Read the csv file and only extract three required columns
    df_list.append(pd.read_csv(os.path.join(input_folder, csv_file)))
    
df = pd.concat(df_list, ignore_index=True)
df.dropna(subset=['VWAP'], inplace=True)
regression_setup = setup(data=df, target='VWAP', train_size=0.7)
best = compare_models()
scores = []
for i, df_ in enumerate(df_list):
    df_.dropna(subset=['VWAP'], inplace=True)
    # print(csv_files[i])  # Commented the line
    predict_model(best, data = df_)
    scores.append((csv_files[i][:-4], pull()["RMSE"][0]))
    print()

,Description,Value
0,Session id,2912
1,Target,VWAP
2,Target type,Regression
3,Original data shape,"(774705, 8)"
4,Transformed data shape,"(774705, 8)"
5,Transformed train set shape,"(542293, 8)"
6,Transformed test set shape,"(232412, 8)"
7,Numeric features,5
8,Categorical features,2
9,Preprocess,True


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
ridge,Ridge Regression,2.5582,951.6786,18.0832,0.9869,0.3633,0.5173,0.8530
lar,Least Angle Regression,2.5590,951.7940,18.0838,0.9869,0.3635,0.5178,0.8230
br,Bayesian Ridge,2.5572,951.6811,18.0832,0.9869,0.3630,0.5172,0.8880
en,Elastic Net,2.4300,955.4896,18.1367,0.9868,0.3176,0.4238,5.0970
lr,Linear Regression,2.4416,954.5261,18.1565,0.9867,0.3151,0.4117,2.4300
lasso,Lasso Regression,2.4375,957.8466,18.1744,0.9867,0.3142,0.4085,4.3000
llar,Lasso Least Angle Regression,2.4376,957.8620,18.1745,0.9867,0.3142,0.4086,0.8140
dt,Decision Tree Regressor,0.3110,24070.4922,69.1691,0.8755,0.0034,0.0004,3.8520
gbr,Gradient Boosting Regressor,0.5518,24072.0106,69.5714,0.8755,0.0220,0.0227,20.0340
rf,Random Forest Regressor,0.3598,34812.2514,81.4193,0.8533,0.0035,0.0003,246.7980


AUDCAD_hourly_fractal_dimension_normalized_weight.csv


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Ridge Regression,0.3720,0.3926,0.6266,-181.4421,0.2321,0.3854



AUDCHF_hourly_fractal_dimension_normalized_weight.csv


...

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Ridge Regression,0.8077,2.3233,1.5242,-9.6957,0.2318,0.2214



USDSEK_hourly_fractal_dimension_normalized_weight.csv


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Ridge Regression,0.7769,1.0035,1.0017,0.3505,0.0983,0.0899



USDSGD_hourly_fractal_dimension_normalized_weight.csv


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Ridge Regression,0.8315,1.3872,1.1778,-386.8211,0.3859,0.6230


In [3]:
def generate_currency_pair_classes_report(scores):
    """
    Generate a report of currency pair classes based on scores.

    Args:
        scores (list): A list of tuples containing currency pair scores.

    Returns:
        None
    """
    pair_errors = sorted(scores, key=lambda a: a[1])
    classes = [*["Forecastable"] * 3, *["Partially Forecastable"] * 4, *["Non Forecastable"] * 3]
    data = {"Currency Pairs": [], "Errors": [], "Class": []}

    for i in range(10):
        data["Currency Pairs"].append(pair_errors[i][0])
        data["Errors"].append(pair_errors[i][1])
        data["Class"].append(classes[i])

    output_folder = "C:/Users/aksha/Downloads/data_eng_fin/classification/hourly_data/fd_normalized/weighted_avg_output_folder/report"
    pd.DataFrame(data).to_csv(output_folder + "Currency Pair Classes.csv")
